### Задание 1

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
Наименование вакансии.
Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
Ссылку на саму вакансию.
Сайт, откуда собрана вакансия.
По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [45]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

In [49]:
def hh_parsing(vacancy):

    vacancy_data = []
    
    params = {
        'text': vacancy, \
        'search_field': 'name', \
        'items_on_page': '100', \
        'page': ''
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
    }

    link = 'https://hh.ru/search/vacancy'
       
    html = requests.get(link, params=params, headers=headers)
    
    if html.ok:
        parsed_html = bs(html.text,'html.parser')
        
        page_block = parsed_html.find('div', {'data-qa': 'pager-block'})
        if not page_block:
            last_page = '1'
        else:
            last_page = int(page_block.find_all('a', {'class': 'bloko-button'})[-2].getText())
    
    for page in range(0, last_page):
        params['page'] = page
        html = requests.get(link, params=params, headers=headers)
        
        if html.ok:
            parsed_html = bs(html.text,'html.parser')
            
            vacancy_items = parsed_html.find('div', {'data-qa': 'vacancy-serp__results'}) \
                                        .find_all('div', {'class': 'vacancy-serp-item'})
                
            for item in vacancy_items:
                vacancy_data.append(hh_item_parsing(item))
                
    return vacancy_data

In [50]:
def hh_item_parsing(item):

    vacancy_data = {}
    
    # vacancy_name
    vacancy_name = item.find('div', {'class': 'serp-item__title'}) \
                        .getText() \
                        .replace(u'\xa0', u' ')
    
    vacancy_data['vacancy_name'] = vacancy_name
    
    # company_name
    company_name = item.find('div', {'class': 'vacancy-serp-item__meta-info-company'}) \
                        .find('a') \
                        .getText()
    
    vacancy_data['company_name'] = company_name
    
    #salary
    salary = item.find('div', {'class': 'bloko-header-section-3'})
    if not salary:
        salary_min = None
        salary_max = None
        salary_currency = None
    else:
        salary = salary.getText() \
                        .replace(u'\xa0', u'')
        
        salary = re.split(r'\s|-', salary)
        
        if salary[0] == 'до':
            salary_min = None
            salary_max = int(salary[1])
        elif salary[0] == 'от':
            salary_min = int(salary[1])
            salary_max = None
        else:
            salary_min = int(salary[0])
            salary_max = int(salary[1])            
        salary_currency = salary[2]
        
    vacancy_data['salary_min'] = salary_min
    vacancy_data['salary_max'] = salary_max
    vacancy_data['salary_currency'] = salary_currency
    
    # link
    is_ad = item.find('h3', {'class': 'bloko-header-section-3'}) \
                .getText()
    
    vacancy_link = item.find('div', {'class': 'vacancy-serp-item-body__main-info'}) \
                        .find('a')['href']

    # site
    vacancy_data['site'] = 'hh.ru'
    
    return vacancy_data


In [60]:
def parser_vacancy(vacancy):
        
    vacancy_date = []
    vacancy_date.extend(hh_parsing(vacancy))
    
    df = pd.DataFrame(vacancy_date)

In [62]:
print(parser_vacancy(vacancy))

Empty DataFrame
Columns: []
Index: []
None


In [63]:
df.info

<bound method DataFrame.info of Empty DataFrame
Columns: []
Index: []>